#### Read any input string and parse it (2 marks)

In [1]:
# Define a function to compute the First set for a given symbol
def compute_first(grammar, symbol, first_sets):
    if symbol in first_sets:
        return first_sets[symbol]

    first = set()
    if symbol not in grammar:
        first.add(symbol)
    else:
        for production in grammar[symbol]:
            if production[0] == symbol:
                continue  # Avoid left recursion

            for char in production:
                if char != symbol:
                    first.update(compute_first(grammar, char, first_sets))
                    if char not in grammar or 'epsilon' not in grammar[char]:
                        break
            else:
                first.add('epsilon')

    first_sets[symbol] = first
    return first

# Define a function to compute the Follow set for a given symbol
def compute_follow(grammar, symbol, follow_sets, first_sets):
    if symbol in follow_sets:
        return follow_sets[symbol]

    follow = set()
    if symbol == grammar['S'][0][0]:  # If it's the start symbol
        follow.add('$')  # Add end-of-input marker

    for left_symbol in grammar:
        for production in grammar[left_symbol]:
            for idx, right_symbol in enumerate(production):
                if right_symbol == symbol:
                    next_symbol = None

                    # Find the next symbol in the production
                    if idx + 1 < len(production):
                        next_symbol = production[idx + 1]

                    if next_symbol:
                        # Add First(next_symbol) to Follow(symbol)
                        follow.update(compute_first(grammar, next_symbol, first_sets))

                        # If First(next_symbol) contains epsilon, add Follow(left_symbol) to Follow(symbol)
                        if 'epsilon' in compute_first(grammar, next_symbol, first_sets):
                            follow.update(compute_follow(grammar, left_symbol, follow_sets, first_sets))
                    else:
                        # If it's the last symbol in the production, add Follow(left_symbol) to Follow(symbol)
                        follow.update(compute_follow(grammar, left_symbol, follow_sets, first_sets))

    follow_sets[symbol] = follow
    return follow

# Define a function to create the LL(1) parsing table
def create_ll1_parsing_table(grammar):
    first_sets = {}
    follow_sets = {}
    parsing_table = {}

    for non_terminal in grammar:
        compute_first(grammar, non_terminal, first_sets)
        compute_follow(grammar, non_terminal, follow_sets, first_sets)

    for non_terminal in grammar:
        for production in grammar[non_terminal]:
            first = compute_first(grammar, production[0], first_sets)
            for terminal in first:
                if terminal:
                    if non_terminal not in parsing_table:
                        parsing_table[non_terminal] = {}
                    if terminal in parsing_table[non_terminal]:
                        return None  # Conflict in parsing table
                    parsing_table[non_terminal][terminal] = production

            if 'epsilon' in first:
                follow = follow_sets[non_terminal]
                for terminal in follow:
                    if terminal:
                        if non_terminal not in parsing_table:
                            parsing_table[non_terminal] = {}
                        if terminal in parsing_table[non_terminal]:
                            return None  # Conflict in parsing table
                        parsing_table[non_terminal][terminal] = production

    return parsing_table

# Define a function to parse the input string using the LL(1) parsing table
def parse_input_string(input_string, parsing_table):
    stack = ['$']
    stack.append('S')  # Start symbol

    i = 0  # Input string index
    while stack:
        top = stack.pop()
        if top == '$':
            if input_string[i] == '$':  # Successfully parsed the input string
                return True
            else:
                return False
        elif top.isalpha() or top == 'epsilon':
            if top == input_string[i]:
                i += 1
            elif top == 'epsilon':
                continue  # Epsilon matches with nothing
            else:
                return False
        elif top in parsing_table and input_string[i] in parsing_table[top]:
            production = parsing_table[top][input_string[i]]
            if production[0] != 'epsilon':
                for symbol in reversed(production):
                    stack.append(symbol)
        else:
            return False

    return False

# Get the LL(1) grammar from the user
user_grammar_input = input("Enter the LL(1) grammar (e.g., S -> Aa | b, A -> Ac | epsilon): ")

# Parse the user input to create the grammar dictionary
ll1_grammar = {}
productions = user_grammar_input.split(', ')
for production in productions:
    left, right = production.split(' -> ')
    if left not in ll1_grammar:
        ll1_grammar[left] = []
    right_symbols = right.split(' ')
    if 'epsilon' in right_symbols:
        right_symbols.remove('epsilon')
        right_symbols.append('epsilon')  # Move epsilon to the end if present
    ll1_grammar[left].append(right_symbols)

# Create the LL(1) parsing table
parsing_table = create_ll1_parsing_table(ll1_grammar)

if parsing_table is None:
    print("The provided grammar is not LL(1) due to conflicts in the parsing table.")
else:
    # Get the input string from the user
    input_string = input("Enter the input string (e.g., baa$): ")

    # Add end-of-input marker
    input_string += '$'

    # Parse the input string using the LL(1) parsing table
    if parse_input_string(input_string, parsing_table):
        print("The input string is valid according to the LL(1) grammar.")
    else:
        print("The input string is not valid according to the LL(1) grammar.")


The input string is not valid according to the LL(1) grammar.
